In [1]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.core.rust.model import TriggerType, TimeInForce
from nautilus_trader.model import TradeTick
from nautilus_trader.model import InstrumentId

In [2]:
catalog = ParquetDataCatalog("/home/p/p-dev/nautilus/catalog")
instrument_id = InstrumentId.from_str("APTUSDT-PERP.BINANCE")

In [3]:
venue = BacktestVenueConfig(
    name="BINANCE",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USDT",
    starting_balances=["100 USDT"],
    default_leverage=5,
    leverages={"APTUSDT-PERP.BINANCE": 5}
)

In [4]:
data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=TradeTick,
    instrument_ids=[instrument_id],
    start_time="2024-12-10",
    end_time="2025-01-10"
)

In [5]:
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
              "instrument_id": instrument_id,
              "fast_period": 12,
              "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="DEBUG"),
)

In [6]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.core.rust.common import LogColor
from nautilus_trader.model.events import PositionOpened, PositionChanged, OrderAccepted, OrderFilled
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig



class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 10000
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        self.log.info("Initialization of MACDStrategy")
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )

        # Convenience
        self.position: Position | None = None

    
    def on_start(self):
        self.log.info("Start of MACDStrategy")
        
        self.instrument = self.cache.instrument(self.config.instrument_id)
        self.account = self.portfolio.account(self.instrument.venue)
        self.trade_size = self.instrument.make_qty(self.config.trade_size)

        self.subscribe_trade_ticks(instrument_id=self.config.instrument_id)


    def on_stop(self):
        self.log.info("Stop of MACDStrategy")
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_trade_ticks(instrument_id=self.config.instrument_id)


    def on_position_created(self, event: PositionChanged):
        self.log.info(f"After postition created with qty {event.last_qty} balances locked: " + 
             f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.CYAN)

    
    def on_position_changed(self, event: PositionChanged):
        self.log.info(f"After postition changed with qty {event.last_qty} balances locked: " + 
             f"{self.account.balances_locked()[self.account.base_currency].as_double()}", LogColor.CYAN)

    
    def on_trade_tick(self, tick: TradeTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available
        self.macd.handle_trade_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry()
        self.check_for_exit()


    def check_for_entry(self):
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.submit_order(order)
            
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT
            
            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.submit_order(order)
            

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)


    def on_dispose(self):
        pass  # Do nothing else

In [7]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
    chunk_size=1024*32
)

In [8]:
from nautilus_trader.backtest.results import BacktestResult


node = BacktestNode(configs=[config])

results: list[BacktestResult] = node.run()

2025-04-13T19:55:52.792528448Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-13T19:55:52.792555379Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-04-13T19:55:52.792557183Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-04-13T19:55:52.792558116Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2025. All rights reserved.
2025-04-13T19:55:52.792559162Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-04-13T19:55:52.792560485Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-04-13T19:55:52.792561106Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-04-13T19:55:52.792563639Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-04-13T19:55:52.792564768Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2025-04-13T19:55

2024-12-10T00:00:05.048000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=40.32199995, margin=608.71560000, free=-568.39360005, currency=USDT)
2024-12-10T00:20:05.689000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=40.29043235, margin=608.71560000, free=-568.42516765, currency=USDT)
2024-12-10T00:46:14.322000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=40.18799283, margin=608.71560000, free=-568.52760717, currency=USDT)
2024-12-10T01:24:40.709000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=40.21566833, margin=608.71560000, free=-568.49993167, currency=USDT)
2024-12-10T02:07:18.254000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=39.91709621, margin=608.71560000, free=-568.79850379, currency=USDT)
2024-12-10T02:43:03.721000000Z [ERROR] B

.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93380,14.7,SELLER,245140984,1733788805024000000)
2024-12-10T00:00:05.024000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93370,0.5,SELLER,245140985,1733788805024000000)
2024-12-10T00:00:05.024000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93360,0.9,SELLER,245140986,1733788805024000000)
2024-12-10T00:00:05.024000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93350,0.5,SELLER,245140987,1733788805024000000)
2024-12-10T00:00:05.024000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93330,0.9,SELLER,245140988,1733788805024000000)
2024-12-10T00:00:05.024000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.93300,1.4,SELLER,245140989,17337888050

2024-12-10T03:09:23.308000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=39.89255299, margin=608.71560000, free=-568.82304701, currency=USDT)
2024-12-10T03:25:29.327000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=39.82908235, margin=608.71560000, free=-568.88651765, currency=USDT)
2024-12-10T03:43:16.672000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=39.82396486, margin=608.71560000, free=-568.89163514, currency=USDT)
2024-12-10T04:08:28.423000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=39.82054249, margin=608.71560000, free=-568.89505751, currency=USDT)
2024-12-10T04:38:12.543000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=39.59199307, margin=608.71560000, free=-569.12360693, currency=USDT)
2024-12-10T05:19:22.762000000Z [ERROR] B

-10T00:20:05.689000000Z [DEBUG] BACKTESTER-001.Cache: Added MarketOrder(SELL 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-002005-001-000-2, venue_order_id=None, position_id=None, tags=None)
2024-12-10T00:20:05.689000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(SELL 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-002005-001-000-2, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T00:20:05.689000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-002005-001-000-2, account_id=BINANCE-001, ts_event=1733790005689000000)
2024-12-10T00:20:05.689000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Market: bid=5.3 @ 11.91230, ask=5.3 @ 11.91230, last=11.91230
2024-12-10T00:20:05.689000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Applying fills to Marke

...

2024-12-10T18:07:25.030000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.66013028, margin=608.71560000, free=-570.05546972, currency=USDT)
2024-12-10T18:33:06.463000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.65467793, margin=608.71560000, free=-570.06092207, currency=USDT)


stEngine: -----------------------------------------------------------------
2024-12-10T12:40:11.406000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.43860,29.3,SELLER,245763568,1733834411406000000)
2024-12-10T12:40:11.406000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-124011-001-000-20, side=SELL, type=MARKET, quantity=10_000.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-10T12:40:11.406000000Z [DEBUG] BACKTESTER-001.Cache: Added MarketOrder(SELL 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-124011-001-000-20, venue_or

2024-12-10T19:14:13.385000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.64743985, margin=608.71560000, free=-570.06816015, currency=USDT)
2024-12-10T20:05:10.749000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.54822777, margin=608.71560000, free=-570.16737223, currency=USDT)
2024-12-10T20:59:59.263000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.47114142, margin=608.71560000, free=-570.24445858, currency=USDT)


ngine: Run ID:         4c97398c-702e-41d7-a6ef-14753fed9eb3
2024-12-10T13:39:04.906000000Z [INFO] BACKTESTER-001.BacktestEngine: Run started:    2025-04-13T19:55:53.408935000Z
2024-12-10T13:39:04.906000000Z [INFO] BACKTESTER-001.BacktestEngine: Backtest start: 2024-12-10T00:00:04.998000000Z
2024-12-10T13:39:04.906000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch start:    2024-12-10T14:11:09.185000000Z
2024-12-10T13:39:04.906000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch end:      2024-12-10T14:41:11.180000000Z
2024-12-10T13:39:04.906000000Z [INFO] BACKTESTER-001.BacktestEngine: -----------------------------------------------------------------
2024-12-10T14:11:09.185000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.65600,2.1,SELLER,245861872,1733839869185000000)
2024-12-10T14:11:09.185000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-141109-0

2024-12-10T21:37:38.849000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.46593060, margin=608.71560000, free=-570.24966940, currency=USDT)


In [9]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

CKTESTER-001.BacktestEngine: Backtest start: 2024-12-10T00:00:04.998000000Z
2024-12-10T14:41:11.183000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch start:    2024-12-10T15:05:33.298000000Z
2024-12-10T14:41:11.183000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch end:      2024-12-10T15:30:53.331000000Z
2024-12-10T14:41:11.183000000Z [INFO] BACKTESTER-001.BacktestEngine: -----------------------------------------------------------------
2024-12-10T15:05:33.298000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.45470,4.2,BUYER,245927408,1733843133298000000)
2024-12-10T15:05:33.298000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-150533-001-000-25, side=SELL, type=MARKET, quantity=10_000.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type

2024-12-10T23:11:39.758000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.17021872, margin=608.71560000, free=-570.54538128, currency=USDT)
2024-12-11T00:30:17.564000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.16733392, margin=608.71560000, free=-570.54826608, currency=USDT)
2024-12-11T01:06:21.562000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=38.16221737, margin=608.71560000, free=-570.55338263, currency=USDT)
2024-12-11T01:36:13.460000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=37.99338851, margin=608.71560000, free=-570.72221149, currency=USDT)
2024-12-11T02:19:21.219000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=37.98829266, margin=608.71560000, free=-570.72730734, currency=USDT)
2024-12-11T03:07:09.504000000Z [ERROR] B

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20241210-000005-001-000-1,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-001,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-002,MARKET,SELL,10000.0,...,None,None,None,None,None,None,None,a99b3496-e36c-4178-a417-deaa7fbfbeef,2024-12-10 00:00:05.048000+00:00,2024-12-10 00:00:05.048000+00:00
O-20241210-002005-001-000-2,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-002,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-003,MARKET,SELL,10000.0,...,None,None,None,None,None,None,None,334ce321-80a8-4e6a-ab8d-62801d71f8fa,2024-12-10 00:20:05.689000+00:00,2024-12-10 00:20:05.689000+00:00
O-20241210-004614-001-000-3,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-003,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-004,MARKET,BUY,10000.0,...,None,None,None,None,None,None,None,28eae577-e451-4286-836e-2c212cf56a82,2024-12-10 00:46:14.322000+00:00,2024-12-10 00:46:14.322000+00:00
O-20241210-012440-001-000-4,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-004,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-005,MARKET,BUY,10000.0,...,None,None,None,None,None,None,None,9131aba0-f7a8-4a5f-9351-0143fcdf3467,2024-12-10 01:24:40.709000+00:00,2024-12-10 01:24:40.709000+00:00
O-20241210-020718-001-000-5,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-005,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-006,MARKET,SELL,10000.0,...,None,None,None,None,None,None,None,6f250112-17b5-4324-80d7-f935cae141fe,2024-12-10 02:07:18.254000+00:00,2024-12-10 02:07:18.254000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
O-20241211-182156-001-000-60,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-060,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-061,MARKET,BUY,10000.0,...,None,None,None,None,None,None,None,fda5af50-25b0-4e36-8d53-043e53ed8415,2024-12-11 18:21:56.089000+00:00,2024-12-11 18:21:56.089000+00:00
O-20241211-201438-001-000-61,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-061,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-062,MARKET,BUY,10000.0,...,None,None,None,None,None,None,None,462c5855-c2e6-4c21-b926-f7b1bf4a0718,2024-12-11 20:14:38.742000+00:00,2024-12-11 20:14:38.742000+00:00
O-20241211-222529-001-000-62,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-1-062,APTUSDT-PERP.BINANCE-MACDStrategy-000,BINANCE-001,BINANCE-1-063,MARKET,BUY,10000.0,...,None,None,None,None,None,None,None,4136d270-b84e-405b-b028-ab375eab7515,2024-12-11 22:25:29.525000+00:00,2024-12-11 22:25:29.525000+00:00


1m2024-12-10T17:44:33.685000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch start:    2024-12-10T18:07:25.030000000Z
2024-12-10T17:44:33.685000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch end:      2024-12-10T18:33:06.463000000Z
2024-12-10T17:44:33.685000000Z [INFO] BACKTESTER-001.BacktestEngine: -----------------------------------------------------------------
2024-12-10T18:07:25.030000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,10.82730,1.0,BUYER,246222320,1733854045030000000)
2024-12-10T18:07:25.030000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-180725-001-000-34, side=SELL, type=MARKET, quantity=10_000.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_i

2024-12-11T10:45:18.800000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=62.11734167, margin=608.71560000, free=-546.59825833, currency=USDT)
2024-12-11T12:06:29.364000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=62.31441389, margin=608.71560000, free=-546.40118611, currency=USDT)
2024-12-11T13:15:08.850000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=62.27641250, margin=608.71560000, free=-546.43918750, currency=USDT)


PTUSDT-PERP.BINANCE MARKET GTC, status=SUBMITTED, client_order_id=O-20241210-191413-001-000-36, venue_order_id=None, position_id=None, tags=None), venue_position_id=None, position=Position(SHORT 10_000.0 APTUSDT-PERP.BINANCE, id=APTUSDT-PERP.BINANCE-MACDStrategy-000), fills=[(Price(11.13550), Quantity(1.3))]
2024-12-10T19:14:13.385000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241210-191413-001-000-36') with VenueOrderId('BINANCE-1-036')
2024-12-10T19:14:13.385000000Z [DEBUG] BACKTESTER-001.Portfolio: Calculated PnLs: []
2024-12-10T19:14:13.385000000Z [INFO] BACKTESTER-001.BacktestEngine: Added 32_768 data elements
2024-12-10T19:14:13.385000000Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2024-12-10T19:14:13.385000000Z [INFO] BACKTESTER-001.BacktestEngine:  BACKTEST RUN
2024-12-10T19:14:13.385000000Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2024-1

2024-12-11T13:47:03.996000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=60.81473060, margin=608.71560000, free=-547.90086940, currency=USDT)
2024-12-11T14:39:12.200000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=60.55732363, margin=608.71560000, free=-548.15827637, currency=USDT)
2024-12-11T15:18:49.818000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=54.39282097, margin=608.71560000, free=-554.32277903, currency=USDT)


hingEngine(BINANCE): Processing TradeTick(APTUSDT-PERP.BINANCE,11.42020,13.5,SELLER,246353392,1733864399263000000)
2024-12-10T20:59:59.263000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241210-205959-001-000-38, side=SELL, type=MARKET, quantity=10_000.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-10T20:59:59.263000000Z [DEBUG] BACKTESTER-001.Cache: Added MarketOrder(SELL 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-205959-001-000-38, venue_order_id=None, position_id=None, tags=None)
2024-12-10T20:59:59.263000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=Mar

In [10]:
engine.trader.generate_positions_report()

2024-12-11T15:52:42.073000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=51.90118951, margin=608.71560000, free=-556.81441049, currency=USDT)


USDT-PERP.BINANCE, client_order_id=O-20241210-231139-001-000-40, side=SELL, type=MARKET, quantity=10_000.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-10T23:11:39.758000000Z [DEBUG] BACKTESTER-001.Cache: Added MarketOrder(SELL 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-231139-001-000-40, venue_order_id=None, position_id=None, tags=None)
2024-12-10T23:11:39.758000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(SELL 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241210-231139-001-000-40, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-10T23:11:39.758000000Z [IN

2024-12-11T16:22:28.418000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=42.67693161, margin=608.71560000, free=-566.03866839, currency=USDT)
2024-12-11T16:45:17.563000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=42.05728658, margin=608.71560000, free=-566.65831342, currency=USDT)
2024-12-11T17:24:39.333000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=28.97669134, margin=608.71560000, free=-579.73890866, currency=USDT)


,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
APTUSDT-PERP.BINANCE-MACDStrategy-000,BACKTESTER-001,MACDStrategy-000,APTUSDT-PERP.BINANCE,BINANCE-001,O-20241210-000005-001-000-1,None,SELL,SHORT,10000.0,10000.0,1733788805048000000,2024-12-10 00:00:05.048000+00:00,1733788805048000000,<NA>,None,11.9356,None,[59.67800005 USDT],0.0,-59.67800005 USDT


==
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine:  BACKTEST RUN
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine: Run config ID:  d3366cf80bf664da0ca56455073e113e722caac7f13f9fafb9635281547bd999
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine: Run ID:         4c97398c-702e-41d7-a6ef-14753fed9eb3
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine: Run started:    2025-04-13T19:55:53.408935000Z
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine: Backtest start: 2024-12-10T00:00:04.998000000Z
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch start:    2024-12-11T02:19:21.219000000Z
2024-12-11T01:36:13.460000000Z [INFO] BACKTESTER-001.BacktestEngine: Batch end:      2024-12-11T03:07:09.504000000Z
2024-12-11T01:36:13.460000000Z [INFO] BACKT

2024-12-11T18:21:56.089000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=28.30480724, margin=608.71560000, free=-580.41079276, currency=USDT)
2024-12-11T20:14:38.742000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=7.64348190, margin=608.71560000, free=-601.07211810, currency=USDT)
2024-12-11T22:25:29.525000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=6.92234967, margin=608.71560000, free=-601.79325033, currency=USDT)


m2024-12-11T04:35:39.985000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(BUY 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241211-043539-001-000-46, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-11T04:35:39.985000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241211-043539-001-000-46, account_id=BINANCE-001, ts_event=1733891739985000000)
2024-12-11T04:35:39.985000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Market: bid=8.5 @ 11.61300, ask=8.5 @ 11.61300, last=11.61300
2024-12-11T04:35:39.985000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Applying fills to MarketOrder(BUY 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=SUBMITTED, client_order_id=O-20241211-043539-001-000-46, venue_order_id=None, position_id=None, tags=None), venue_position_id=None, position=Position(SHORT 10_000.0 APTUSDT

2024-12-12T00:25:03.723000000Z [ERROR] BACKTESTER-001.BacktestEngine: Stopping backtest from AccountMarginExceeded(balance=6.46981759, margin=608.71560000, free=-602.24578241, currency=USDT)


11T08:03:42.373000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderInitialized(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241211-080342-001-000-48, side=BUY, type=MARKET, quantity=10_000.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-11T08:03:42.373000000Z [DEBUG] BACKTESTER-001.Cache: Added MarketOrder(BUY 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241211-080342-001-000-48, venue_order_id=None, position_id=None, tags=None)
2024-12-11T08:03:42.373000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(BUY 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241211-080342-001-000-48, venu

2024-12-12T01:08:19.843000000Z [ERROR] BACKTESTER-001.BacktestNode: Error running backtest: `total` amount was negative


In [11]:
engine.trader.generate_account_report(Venue("BINANCE"))

2024-12-11T09:06:30.255000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, client_order_id=O-20241211-090630-001-000-49, account_id=BINANCE-001, ts_event=1733907990255000000)
2024-12-11T09:06:30.255000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Market: bid=12.3 @ 11.71870, ask=12.3 @ 11.71870, last=11.71870
2024-12-11T09:06:30.255000000Z [DEBUG] BACKTESTER-001.OrderMatchingEngine(BINANCE): Applying fills to MarketOrder(BUY 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=SUBMITTED, client_order_id=O-20241211-090630-001-000-49, venue_order_id=None, position_id=None, tags=None), venue_position_id=None, position=Position(SHORT 10_000.0 APTUSDT-PERP.BINANCE, id=APTUSDT-PERP.BINANCE-MACDStrategy-000), fills=[(Price(11.71870), Quantity(12.3))]
2024-12-11T09:06:30.255000000Z [DEBUG] BACKTESTER-001.Cache: Indexed ClientOrderId('O-20241211-090630-001-000-49') with VenueOrderId('BINANCE-1-049')
2024-12-11T09:06:30.255000000Z [DEBU

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2024-12-10 00:00:04.998000+00:00,100.00000000,0E-8,100.00000000,USDT,BINANCE-001,MARGIN,USDT,[],True,{}
2024-12-10 00:00:05.048000+00:00,99.99403215,0E-8,99.99403215,USDT,BINANCE-001,MARGIN,USDT,[],False,{}
2024-12-10 00:00:05.048000+00:00,40.32199995,0.06087207,40.26112788,USDT,BINANCE-001,MARGIN,USDT,"[{'type': 'MarginBalance', 'initial': '0E-8', ...",False,{}


antity=10_000.0, time_in_force=GTC, post_only=False, reduce_only=False, quote_quantity=False, options={}, emulation_trigger=NO_TRIGGER, trigger_instrument_id=None, contingency_type=NO_CONTINGENCY, order_list_id=None, linked_order_ids=None, parent_order_id=None, exec_algorithm_id=None, exec_algorithm_params=None, exec_spawn_id=None, tags=None)
2024-12-11T10:45:18.800000000Z [DEBUG] BACKTESTER-001.Cache: Added MarketOrder(BUY 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241211-104518-001-000-50, venue_order_id=None, position_id=None, tags=None)
2024-12-11T10:45:18.800000000Z [INFO] BACKTESTER-001.MACDStrategy: [CMD]--> SubmitOrder(order=MarketOrder(BUY 10_000.0 APTUSDT-PERP.BINANCE MARKET GTC, status=INITIALIZED, client_order_id=O-20241211-104518-001-000-50, venue_order_id=None, position_id=None, tags=None), position_id=None)
2024-12-11T10:45:18.800000000Z [INFO] BACKTESTER-001.MACDStrategy: <--[EVT] OrderSubmitted(instrument_id=APTUSDT-PERP.BINANCE, 